In [24]:
import torch
import numpy as np
import pandas as pd

In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [11]:
outfile = open("../models/mimic-iii/raw_text_questions.txt", 'r')
# for entry in openMessages:
corpus = outfile.read().split("\n")

corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

In [23]:
ce = corpus_embeddings.cpu().numpy()
np.savetxt("../models/mimic-iii/corpus.csv", ce, delimiter=',')

In [43]:
query = "what are the methods of consumption of clotrimazole?"
top_k = min(10, len(corpus))

query_embedding = model.encode(query, convert_to_tensor=True).to('cpu').to(torch.float32)

corpus_embeddings = torch.tensor(pd.read_csv("../models/mimic-iii/corpus.csv").values).to(torch.float32)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 10 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    print(corpus[idx], "(Score: {:.4f})".format(score))

"""
# Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
    print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
"""





Query: what are the methods of consumption of clotrimazole?

Top 10 most similar sentences in corpus:
what are the ways to consume sterile water? (Score: 1.0000)
what is the intake method of the methotrexate sodium p.f.? (Score: 0.9595)
what is the ingesting method of nicotine lozenge? (Score: 0.9588)
what are the ways to consume caspofungin? (Score: 0.9553)
tell me the intake method for sirolimus? (Score: 0.9547)
what is the ingesting method of testosterone 5mg patch? (Score: 0.9547)
tell me the intake method for readi-cat 2 (barium sulfate 2% suspension)? (Score: 0.9546)
what is the ingesting method of methimazole? (Score: 0.9534)
what are the methods of consumption of insulin human nph? (Score: 0.9525)
what are the ways to consume prochlorperazine maleate? (Score: 0.9488)


'\n# Alternatively, we can also use util.semantic_search to perform cosine similarty + topk\nhits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)\nhits = hits[0]      #Get the hits for the first query\nfor hit in hits:\n    print(corpus[hit[\'corpus_id\']], "(Score: {:.4f})".format(hit[\'score\']))\n'

In [42]:
top_results

torch.return_types.topk(
values=tensor([0.9383, 0.9317, 0.9288, 0.9278, 0.9278, 0.9265, 0.9241, 0.9226, 0.9215,
        0.9191], dtype=torch.float32),
indices=tensor([6443, 6463, 6466, 6472, 6469, 6458, 6454, 6442, 6448, 6445]))